In [22]:
import numpy as np
import pandas as pd 

import torch 
import torch.nn as nn


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import auc, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, precision_recall_curve

import os

In [ ]:
best_model_path = f"vibtcr/data/result/NAbest"
base_path = 'vibtcr/data/new_split/pep+cdr3b/'
embed_base_path = 'vibtcr/data/embedNA/val/' 

In [24]:
# load model
class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes=[512, 512, 512, 256, 256, 256], dropout=0.2):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.dropout = dropout
        
        layers = []
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(nn.BatchNorm1d(hidden_sizes[0]))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        
        for i in range(len(hidden_sizes) - 1):
            layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            layers.append(nn.BatchNorm1d(hidden_sizes[i+1]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [25]:
model = MLP(input_size = 768 + 480, output_size = 2, hidden_sizes = [32], dropout = 0.3)
# model.load_state_dict(torch.load(os.path.join(model_path, "best_mol-esm_0.pth")))

In [26]:
def validation_split(neg_generate_mode, split_id):
    train_df_path = os.path.join(DATA_BASE, "train", neg_generate_mode, f"train-{split_id}.csv")

    test_df_path = os.path.join(DATA_BASE, "test", neg_generate_mode, f"test-{split_id}.csv")
    
    train_df = make_df(train_df_path)
    test_df = make_df(test_df_path)
    
    num_samples = train_df.shape[0]
    num_validation = int(num_samples * validation_ratio)

    from collections import Counter
    peptide_count = Counter(train_df['peptide'])
    peptide_count_len = len(peptide_count)
    peptide_perm = np.random.RandomState(seed=42).permutation(peptide_count_len)

    c = 0
    selected_peptide = []
    for i in peptide_perm:
        selected_peptide.append(peptide_count.most_common()[i][0])
        c += peptide_count.most_common()[i][1]
        if c > num_validation:
            break

    new_train_df = train_df[~train_df['peptide'].isin(selected_peptide)]
    validation_df = train_df[train_df['peptide'].isin(selected_peptide)]

    check_unseen(new_train_df, validation_df)
    draw_bar(train_df.shape[0], validation_df.shape[0], test_df.shape[0], split_id)
    

    return new_train_df, validation_df, test_df


In [ ]:
def load_data(dataset_index):
    print(f"Processing dataset {dataset_index}...")
    
    df_train = pd.read_csv(f'{base_path}/train/only-neg-assays/train-{dataset_index}.csv', low_memory=False)
    df_val = pd.read_csv(f'{base_path}/validation/only-neg-assays/validation-{dataset_index}.csv', low_memory=False)
    df_test = pd.read_csv(f'{base_path}/test/only-neg-assays/test-{dataset_index}.csv', low_memory=False)
    train_pep_mol = np.load(f'{embed_base_path}/train{dataset_index}_pep_mol.npy')
    val_pep_mol = np.load(f'{embed_base_path}/val{dataset_index}_pep_mol.npy')
    test_pep_mol = np.load(f'{embed_base_path}/test{dataset_index}_pep_mol.npy')
    train_cdr3 = np.load(f'{embed_base_path}/train{dataset_index}_CDR3_esm.npy')
    val_cdr3 = np.load(f'{embed_base_path}/val{dataset_index}_CDR3_esm.npy')
    test_cdr3 = np.load(f'{embed_base_path}/test{dataset_index}_CDR3_esm.npy')

    X_train = np.column_stack((train_pep_mol, train_cdr3))
    X_val = np.column_stack((val_pep_mol, val_cdr3))
    X_test = np.column_stack((test_pep_mol, test_cdr3))

    y_train = df_train['label']
    y_val = df_val['label']
    y_test = df_test['label']
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [28]:
model = model.to("cuda:0")

In [29]:
# %%
def get_representations(sequence, tokenizer, embed_model):
    sequence_representations = []
    for protein in sequence:
        inputs = tokenizer(protein, return_tensors="pt")
        inputs = inputs.to(device)
        
        outputs = embed_model(**inputs, output_hidden_states = True)

        last_hidden_states = outputs.last_hidden_state
        sequence_representation = last_hidden_states[0].mean(dim = 0)
        sequence_representations.append(sequence_representation)

    return torch.stack(sequence_representations, dim = 0)


def get_embeddings(df, save_path, feature, split_set, batch_size = 1000):
    feat_seq = df[feature]

    result = None

    if feature == "tcrb":
        tokenizer = tcr_tokenizer
        embed_model = tcr_embed_model
        for k in trange(feat_seq.shape[0] // batch_size + 1):
            embeddings = get_representations(feat_seq[k * batch_size: (k + 1) * batch_size], tokenizer, embed_model)
            embeddings = embeddings.detach().cpu().numpy()
            if k == 0:
                result = embeddings
            else:
                result = np.vstack([result, embeddings])
        
    elif feature == "peptide":
        tokenizer = peptide_tokenizer
        embed_model = peptide_embed_model
        
        peptide_uniq = list(feat_seq.unique())

        if args.pretrain_peptide_name == "SMILES_BERT":
            inputs = tokenizer(peptide_uniq, return_tensors="pt", truncation=True, padding=True)
            inputs = inputs.to(device)
            outputs = embed_model(**inputs, output_hidden_states = True)
            embeddings = outputs.hidden_states[0].mean(dim = 0)

        else:
            embeddings = get_representations(peptide_uniq, tokenizer, embed_model)

        embeddings = embeddings.detach().cpu().numpy()
        result = embeddings

    if split_set == "train":
        save_path = os.path.join(save_path, f"train-{args.split_id}." + feature + ".npy")
    elif split_set == "validation":
        save_path = os.path.join(save_path, f"validation-{args.split_id}." + feature + ".npy")
    elif split_set == "test":
        save_path = os.path.join(save_path, f"test-{args.split_id}." + feature + ".npy")
    else:
        RaiseError("split_set should be one of train, validation, test")

    np.save(save_path, result)
    print(f"embeddings saved to ", save_path)

In [30]:
def get_diff_ratio(seen_ratio = 0):
    
    num_rows_to_replace = int(np.ceil(X_test.shape[0] * seen_ratio))
    rows_to_replace = np.random.choice(X_test.shape[0], size = num_rows_to_replace, replace=False)
    rows_to_place = np.random.choice(X_train.shape[0], size = num_rows_to_replace, replace=False)

    X_new_test = X_test.copy()
    X_new_test[rows_to_replace, :] = X_new_train[rows_to_place, :]

    y_new_test = y_test.copy()
    y_new_test[rows_to_replace] = y_new_train[rows_to_place]


    return X_new_test, y_new_test

In [31]:
def get_result():

    X_new_test, y_new_test = get_diff_seen_ratio(seen_ratio)
    y_new_pred = model(torch.from_numpy(X_new_test).to(device))
    test_probabilities = torch.softmax(y_new_pred, dim=1)[:, 1].detach().cpu().numpy()
    test_predictions = [1 if prob > 0.5 else 0 for prob in test_probabilities]
    test_auc = roc_auc_score(y_new_test, test_probabilities)
    precision, recall, _ = precision_recall_curve(y_new_test, test_probabilities)

    metrics = {
            'AUROC': test_auc,
            'Accuracy': accuracy_score(y_new_test, test_predictions),
            'Recall': recall_score(y_new_test, test_predictions),
            'Precision': precision_score(y_new_test, test_predictions),
            'F1 score': f1_score(y_new_test, test_predictions),
            'AUPR': auc(recall, precision),
        }

    result_df = pd.DataFrame({
            'score': list(metrics.values()),
            'metrics': list(metrics.keys()),
            'experiment': [dataset_index] * len(metrics)
        })
    
    return result_df

In [32]:
def evaluate_and_save(model, dataset_index, best_model_path):
    X_train, y_train, X_val, y_val, X_test, y_test = load_data(dataset_index)
    model = MLP(input_size = 748 + 500, output_size = 2, hidden_sizes = [32], dropout = 0.3)
    model.load_state_dict(torch.load(os.path.join(best_model_path, f"best_mol-esm_{dataset_index}.pth")))
    model = model.to("cuda:0")
    model.eval()
    # with torch.no_grad():
    #     test_probabilities = []
        # y_true_test = []
        # test_running_loss = 0.0
        # for test_inputs, test_targets in test_loader:
        #     test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)
        #     test_outputs = model(test_inputs.float())
        #     test_loss = criterion(test_outputs, test_targets)
        #     test_running_loss += test_loss.item() * test_inputs.size(0)
        #     test_probabilities.extend(torch.softmax(test_outputs, dim=1)[:, 1].cpu().numpy())
            # y_true_test.extend(test_targets.cpu().numpy())

    y_pred = model(torch.from_numpy(X_test).to("cuda:0"))
    test_probabilities = torch.softmax(y_pred, dim=1)[:, 1].detach().cpu().numpy()
    y_true_test = y_test
    # test_loss = test_running_loss / len(test_loader.dataset)    
    test_auc = roc_auc_score(y_true_test, test_probabilities)    
    test_predictions = [1 if prob > 0.5 else 0 for prob in test_probabilities]
    precision, recall, _ = precision_recall_curve(y_true_test, test_probabilities)
    
    metrics = {
            'AUROC': test_auc,
            'Accuracy': accuracy_score(y_true_test, test_predictions),
            'Recall': recall_score(y_true_test, test_predictions),
            'Precision': precision_score(y_true_test, test_predictions),
            'F1 score': f1_score(y_true_test, test_predictions),
            'AUPR': auc(recall, precision),
        }

    result_df = pd.DataFrame({
            'score': list(metrics.values()),
            'metrics': list(metrics.keys()),
            'experiment': dataset_index
        })
    
    # csv_path = os.path.join(results_dir, f"evaluation_{dataset_index}.csv")
    #result_df.to_csv(csv_path, index=False)

    print(f"\nBest Model Performance and Evaluation of dataset{dataset_index}:")
    for metric, score in metrics.items():
        print(f"{metric}: {score*100:.4f}%")

    return result_df

In [33]:
result_df = pd.DataFrame({
    "score": [],
    "metrics": [],
    "experiment": [] 
})
l = []
for dataset_index in range(5):
    l.append(evaluate_and_save(model, dataset_index, best_model_path))
result_df = pd.concat(l)
result_df

Processing dataset 0...

Best Model Performance and Evaluation of dataset0:
AUROC: 89.2157%
Accuracy: 97.6383%
Recall: 99.6219%
Precision: 98.0015%
F1 score: 98.8051%
AUPR: 99.7560%
Processing dataset 1...

Best Model Performance and Evaluation of dataset1:
AUROC: 82.4534%
Accuracy: 91.4882%
Recall: 99.4476%
Precision: 91.9553%
F1 score: 95.5548%
AUPR: 98.1988%
Processing dataset 2...

Best Model Performance and Evaluation of dataset2:
AUROC: 84.8993%
Accuracy: 97.5290%
Recall: 99.5895%
Precision: 97.9226%
F1 score: 98.7490%
AUPR: 99.6134%
Processing dataset 3...

Best Model Performance and Evaluation of dataset3:
AUROC: 99.1751%
Accuracy: 95.2060%
Recall: 100.0000%
Precision: 95.2060%
F1 score: 97.5441%
AUPR: 99.9552%
Processing dataset 4...

Best Model Performance and Evaluation of dataset4:
AUROC: 94.7579%
Accuracy: 99.8713%
Recall: 99.9777%
Precision: 99.8936%
F1 score: 99.9356%
AUPR: 99.9942%


,score,metrics,experiment
0,0.892157,AUROC,0
1,0.976383,Accuracy,0
2,0.996219,Recall,0
3,0.980015,Precision,0
4,0.988051,F1 score,0
5,0.997560,AUPR,0
0,0.824534,AUROC,1
1,0.914882,Accuracy,1
2,0.994476,Recall,1
3,0.919553,Precision,1


In [35]:
stats = result_df.groupby('metrics')['score'].agg(['mean', 'std']).reset_index()
stats

,metrics,mean,std
0,AUPR,0.995035,0.007455
1,AUROC,0.901003,0.068984
2,Accuracy,0.963466,0.031779
3,F1 score,0.981177,0.016638
4,Precision,0.965958,0.030853
5,Recall,0.997274,0.002477


In [49]:
def test_dataset(dataset_index):
    print(f"Processing dataset {dataset_index}...")
    
    df_train = pd.read_csv(f'{base_path}/train/only-neg-assays/train-{dataset_index}.csv', low_memory=False)
    df_val = pd.read_csv(f'{base_path}/validation/only-neg-assays/validation-{dataset_index}.csv', low_memory=False)
    df_test = pd.read_csv(f'{base_path}/test/only-neg-assays/test-{dataset_index}.csv', low_memory=False)

    # return df_train, df_val, df_test
    train_pep_mol = np.load(f'{embed_base_path}/train{dataset_index}_pep_mol.npy')
    val_pep_mol = np.load(f'{embed_base_path}/val{dataset_index}_pep_mol.npy')
    test_pep_mol = np.load(f'{embed_base_path}/test{dataset_index}_pep_mol.npy')
    

    train_cdr3 = np.load(f'{embed_base_path}/train{dataset_index}_CDR3_esm.npy')
    val_cdr3 = np.load(f'{embed_base_path}/val{dataset_index}_CDR3_esm.npy')
    test_cdr3 = np.load(f'{embed_base_path}/test{dataset_index}_CDR3_esm.npy')

    X_train = np.column_stack((train_pep_mol, train_cdr3))
    X_val = np.column_stack((val_pep_mol, val_cdr3))
    X_test = np.column_stack((test_pep_mol, test_cdr3))

    y_train = df_train['label']
    y_val = df_val['label']
    y_test = df_test['label']
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    train_pep_mol = X_train[:, :768]
    train_cdr3 = X_train[:, 768:]
    val_pep_mol = X_val[:, :768]
    val_cdr3 = X_val[:, 768:]
    test_pep_mol = X_test[:, :768]
    test_cdr3 = X_test[:, 768:]
    

    return df_train, df_val, df_test, train_pep_mol, train_cdr3, val_pep_mol, val_cdr3, test_pep_mol, test_cdr3, y_train, y_val, y_test

In [51]:
df_train, df_val, df_test, train_pep_mol, train_cdr3, val_pep_mol, val_cdr3, test_pep_mol, test_cdr3, y_train, y_val, y_test = test_dataset(0)

Processing dataset 0...


In [56]:
all_df = pd.concat([df_train, df_val, df_test])
all_df.shape

(268961, 3)

In [58]:
all_peptide_embed = np.vstack([train_pep_mol, val_pep_mol, test_pep_mol])
all_tcrb_embed = np.vstack([train_cdr3, val_cdr3, test_cdr3])

all_peptide_embed.shape, all_tcrb_embed.shape

((268961, 768), (268961, 480))

In [65]:
all_df["peptide_embed"] = list(all_peptide_embed)
all_df["tcrb_embed"] = list(all_tcrb_embed)

In [67]:
peptide_unique_df = all_df.drop_duplicates(subset='peptide')
tcrb_unique_df = all_df.drop_duplicates(subset='tcrb')

peptide_unique_df.shape, tcrb_unique_df.shape

((1360, 5), (197850, 5))

In [69]:
peptide_embed_dict = dict(zip(peptide_unique_df["peptide"], peptide_unique_df["peptide_embed"]))
tcrb_embed_dict = dict(zip(tcrb_unique_df["tcrb"], tcrb_unique_df["tcrb_embed"]))

In [ ]:
import pickle

with open("tc-hard/meta_data/moleformer/only-neg-assays/peptide_dict.pkl", "wb") as f:
    pickle.dump(peptide_embed_dict, f)

with open("tc-hard/meta_data/moleformer/only-neg-assays/tcrb_dict.pkl", "wb") as f:
    pickle.dump(tcrb_embed_dict, f)

